In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

In [2]:
from datetime import datetime as dt
import requests
# xml 라이브러리를 이용해서 xml 문서 파싱을 하기 위해 import 한다.
from xml.etree.ElementTree import fromstring, ElementTree
from elasticsearch import Elasticsearch
es = Elasticsearch('http://localhost:9200')

서울시 공공와이파이 서비스 개수를 얻어온다.

In [3]:
targetSite = 'http://openapi.seoul.go.kr:8088/746e4968796d6b7639366763624a53/xml/TbPublicWifiInfo/1/1/'
response = requests.get(targetSite)
# print(response.text)
tree = ElementTree(fromstring(response.text)) # 오픈 API가 응답한 XML을 파싱한다.
# print(tree) # <xml.etree.ElementTree.ElementTree object at 0x000001B16D2A9828>
root = tree.getroot() # 파싱된 XML의 root(최상단) 태그(TbPublicWifiInfo)를 얻어온다.
# print(tree.getroot()) # <Element 'TbPublicWifiInfo' at 0x000001B16DCCEC28>
# find('태그이름') 메소드로 파싱된 XML의 특정 태그를 얻어온다.
count = int(root.find('list_total_count').text)
print('{} 현재 서울시가 제공하는 공공와이파이는 모두 {:,d}개 입니다.'.format(dt.now(), count))

2024-06-28 15:40:06.369188 현재 서울시가 제공하는 공공와이파이는 모두 24,587개 입니다.


서울시 공공와이파이 정보를 엘라스틱서치 인덱스에 저장한다.

In [4]:
# 서울 열린데이터 광장에서 오픈 API를 이용해서 한 번에 얻어올 수 있는 최대 데이터의 개수는 1000개 이므로
# 얻어오려는 데이터의 개수가 1000개를 넘는 경우 1000개씩 나눠서 오픈 API를 호출해야 한다.
idx = 0
for i in range(count // 1000 + 1)[:]:
    # 오픈 API에 요청 주소를 만든다.
    start = i * 1000 + 1
    end = start + 999
    # targetSite = 'http://openapi.seoul.go.kr:8088/746e4968796d6b7639366763624a53/xml/TbPublicWifiInfo/1/1/'
    targetSite = 'http://openapi.seoul.go.kr:8088/746e4968796d6b7639366763624a53/xml/TbPublicWifiInfo/{}/{}/'.format(start, end)
    # print(targetSite)
    
    # 오픈 API가 응답한 XML 데이터를 파싱한다.
    response = requests.get(targetSite)
    tree = ElementTree(fromstring(response.text))
    root = tree.getroot()
    
    # 파싱된 XML은 <TbPublicWifiInfo> ~ </TbPublicWifiInfo> 사이에 여러개의 <row> ~ </row>로 구성되어 있다.
    # <row> ~ </row> 사이의 모든 데이터를 반복을 수행하며 얻어와서 엘라스틱서치에 저장할 데이터를 딕셔너리로 만든다.
    for row in root.iter('row'): # root.findall('row')
        gu = row.find('X_SWIFI_WRDOFC').text # 구이름
        place = row.find('X_SWIFI_MAIN_NM').text # 설치장소
        lat = row.find('LAT').text # 위도, y좌표
        lon = row.find('LNT').text # 경도, x좌표
        # print(gu, place, lat, lon)
        
        doc = {
            'gu': gu,
            'place': place,
            # geo_point(위치 정보) 타입은 위도(y좌표), 경도(x좌표)의 순서로 구성해야 한다.
            'location': {
                'lat': lat, # 위도
                'lon': lon  # 경도
            }
        }
        # print(doc)
        
        # 엘라스틱서치에 저장한다.
        idx += 1
        try:
            es.index(index='seoul_wifi', id=idx, body=doc)
        except:
            doc = {
                'gu': gu,
                'place': place,
                'location': {
                    'lat': lon,
                    'lon': lat
                }
            }
            es.index(index='seoul_wifi', id=idx, body=doc)
    # ===== for row
    print('종료: {:5d}, {:5d}'.format(start, end))

종료:     1,  1000
종료:  1001,  2000
종료:  2001,  3000
종료:  3001,  4000
종료:  4001,  5000
종료:  5001,  6000
종료:  6001,  7000
종료:  7001,  8000
종료:  8001,  9000
종료:  9001, 10000
종료: 10001, 11000
종료: 11001, 12000
종료: 12001, 13000
종료: 13001, 14000
종료: 14001, 15000
종료: 15001, 16000
종료: 16001, 17000
종료: 17001, 18000
종료: 18001, 19000
종료: 19001, 20000
종료: 20001, 21000
종료: 21001, 22000
종료: 22001, 23000
종료: 23001, 24000
종료: 24001, 25000
